## Agent Executor From Scratch
In this notebook we will go over how to build a basic agent executor from scratch.

1. Setup
First we need to install the packages required


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain langchain_openai langchainhub tavily-python

In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.4 MB/s eta 0:00:00


## 2. Create the LangChain Agent
###Create an agent using the LangChain library:

 * Create a search tool (using Tavily).
 * Get a prompt from the LangChain Hub.
 * Choose an LLM (language model) like OpenAI.
 * Create an OpenAI functions agent.


In [4]:
import os  # For setting environment variables

# Set API keys directly as environment variables
os.environ["OPENAI_API_KEY"] = ""
os.environ["TAVILY_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ""

# Import necessary packages from LangChain and other tools
from langchain import hub  # For pulling the prompt from the hub
from langchain.agents import create_openai_functions_agent  # For creating the agent
from langchain_community.tools.tavily_search import TavilySearchResults  # Tavily search tool
from langchain_openai.chat_models import ChatOpenAI  # For the OpenAI LLM model

# Define the tools to be used by the agent, in this case, Tavily search with max 1 result
tools = [TavilySearchResults(max_results=1)]

# Pull the prompt configuration from the LangChain hub
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent, using OpenAI's GPT-3.5-turbo model
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)

# Construct the OpenAI Functions agent with the specified LLM, tools, and prompt
agent_runnable = create_openai_functions_agent(llm, tools, prompt)


## 3. Define the Graph State
We now define the graph state. The state for the traditional LangChain agent has a few attributes:

1. input: This is the input string representing the main ask from the user, passed in as input.
2. chat_history: This is any previous conversation messages, also passed in as input.
3. intermediate_steps: This is list of actions and corresponding observations that the agent takes over time. This is updated each iteration of the agent.
4. agent_outcome: This is the response from the agent, either an AgentAction or AgentFinish. The AgentExecutor should finish when this is an AgentFinish, otherwise it should call the requested tools.

In [5]:
import operator  # Provides mathematical operators for use in expressions
from typing import Annotated, TypedDict, Union  # Provides type annotations for enhanced code clarity

from langchain_core.agents import AgentAction, AgentFinish  # Defines actions and final states for agents
from langchain_core.messages import BaseMessage  # Base class for messages exchanged between agents



class AgentState(TypedDict):
    # The input string
    input: str
    # The list of previous messages in the conversation
    chat_history: list[BaseMessage]
    # The outcome of a given call to the agent
    # Needs `None` as a valid type, since this is what this will start as
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # List of actions and corresponding observations
    # Here we annotate this with `operator.add` to indicate that operations to
    # this state should be ADDED to the existing values (not overwrite it)
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

## 4. Define the nodes
We now need to define a few different nodes in our graph. In langgraph, a node can be either a function or a runnable. There are two main nodes we need for this:

   1. The agent: responsible for deciding what (if any) actions to take.
   2. A function to invoke tools: if the agent decides to take an action, this node will then execute that action.

We will also need to define some edges. Some of these edges may be conditional. The reason they are conditional is that based on the output of a node, one of several paths may be taken. The path that is taken is not known until that node is run (the LLM decides).

  1. Conditional Edge: after the agent is called, we should either: a. If the agent said to take an action, then the function to invoke tools should be called b. If the agent said that it was finished, then it should finish
  2. Normal Edge: after the tools are invoked, it should always go back to the agent to decide what to do next
Let's define the nodes, as well as a function to decide how what conditional edge to take.

In [8]:
from langchain_core.agents import AgentFinish

from langgraph.prebuilt.tool_executor import ToolExecutor

# This a helper class we have that is useful for running tools
# It takes in an agent action and calls that tool and returns the result
tool_executor = ToolExecutor(tools)


# Define the agent
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}


# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}


# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"


<ipython-input-8-4679f865450e>:7: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


## 5.  Define the graph
###We can now put it all together and define the graph!
 * Create a new graph and add the nodes.
 * Set the entry point (first node to be called).
 * Add conditional and normal edges.
 * Compile the graph to convert it into a runnable format.

In [9]:
from langgraph.graph import END, StateGraph, START  # Import necessary components for the graph

# Define a new graph with the AgentState
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# Add conditional edges based on the agent's outcome
workflow.add_conditional_edges(
    "agent",  # Node to check condition
    should_continue,  # Function to determine the edge
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        "continue": "action",  # If 'continue', go to the action node
        "end": END,  # If 'end', finish the graph
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Compile the workflow into a runnable application
# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()


## 6. Run the Graph
 * Run the graph with some input data:
 * Use the compiled graph to process inputs.
 * The graph handles inputs, updates the state, and determines actions.
 * Outputs include intermediate steps and the final result.

In [10]:
# Define the input data for the agent
inputs = {"input": "what is the weather in pakistan", "chat_history": []}

# Stream the results of running the app with the input data
for s in app.stream(inputs):
    print(list(s.values())[0])  # Print the result of each step
    print("----")  # Separator for clarity


{'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in Pakistan'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in Pakistan'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in Pakistan"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_5aa43294a1'}, id='run-4bc1a69b-dde7-4c6a-a18c-f81cbf4a599a-0')])}
----
{'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in Pakistan'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in Pakistan'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in Pakistan"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call', 'model

#Summary
 * This process allows you to create a custom agent executor using LangGraph, which is similar to LangChain.
 * Future videos will cover more details on the state graph interface and streaming results.
 * By following these steps, you can build and visualize the execution of a custom agent in LangGraph using LangSmith for observability.bold text


